In [390]:
# Code used for cleaning, cleaning notes contains what I need to change
import pandas as pd

In [391]:
# cg short for cleangames
cg = pd.read_csv('csv/carlsen_games.csv')
cg.head()

,game_id,game_order,event,site,date_played,round,white,black,result,white_elo,...,termination,time_control,utc_date,utc_time,variant,ply_count,date_created,file_name,group_name_color,group_name_result
0,b61b981c-e005-4064-8cc8-6feae75530b3,1,Lichess Titled Arena 6,https://lichess.org/YpffsNPV,2018.08.04,?,BabaRamdev,damnsaltythatsport,1-0,2739,...,Normal,60+0,2018.08.04,20:58:08,Standard,NaN,2021-12-26T15:21:45+0000,damnsaltythatsport.pgn,black,lost
1,913cad09-25de-4c53-8937-17772cd05544,2,Lichess Titled Arena 6,https://lichess.org/TIzxPIZz,2018.08.04,?,damnsaltythatsport,penguingim1,1-0,2863,...,Normal,60+0,2018.08.04,20:56:59,Standard,NaN,2021-12-26T15:21:45+0000,damnsaltythatsport.pgn,white,won
2,62fcaebd-c829-4357-b9eb-1c1c373d6685,3,Lichess Titled Arena 6,https://lichess.org/FAWl09E2,2018.08.04,?,Hawk20,damnsaltythatsport,0-1,2677,...,Normal,60+0,2018.08.04,20:54:40,Standard,NaN,2021-12-26T15:21:45+0000,damnsaltythatsport.pgn,black,won
3,f075f85f-bafc-4d21-8841-cb3017fbd43b,4,Lichess Titled Arena 6,https://lichess.org/cEanOYfi,2018.08.04,?,damnsaltythatsport,opperwezen,1-0,2848,...,Normal,60+0,2018.08.04,20:52:19,Standard,NaN,2021-12-26T15:21:45+0000,damnsaltythatsport.pgn,white,won
4,bb6c0012-3863-4669-9c4e-9d230b720b04,5,Lichess Titled Arena 6,https://lichess.org/31FDS3I7,2018.08.04,?,caching,damnsaltythatsport,1-0,2770,...,Normal,60+0,2018.08.04,20:50:20,Standard,NaN,2021-12-26T15:21:45+0000,damnsaltythatsport.pgn,black,lost


In [392]:
cg.shape

(13442, 31)

In [393]:
# Date and time don't need to be seperated. valuecounts prove that each datetime is unique and can be used to identify games if necessary
cg['datetime'] = cg.utc_date + " " + cg.utc_time
cg.datetime.value_counts()

2018.08.04 20:58:08    1
2020.04.02 03:18:54    1
2020.04.02 03:56:05    1
2020.04.02 03:54:18    1
2020.04.02 03:53:25    1
                      ..
2020.11.06 22:56:09    1
2020.11.06 22:54:11    1
2020.10.27 17:26:28    1
2020.10.27 17:24:28    1
2020.06.06 17:11:27    1
Name: datetime, Length: 13442, dtype: int64

In [394]:
# Deleting game_order, site, event, rating_diff(s), title(s), eco, termination, variant, file_name, date_created because I don't need it.
# Deleting ply_count because it's empty.
# Deleting round because it only has question marks.
# Deleting date_played because it's the exact same as utc_date.
# Deleting winner, winner_elo, loser, loser_elo because it's black, black_elo, white, white_elo rearranged.
# Deleting utc_date and utc_time because I merged them into datetime.
# Remaning group_name_result to magnus_result and group_name_color to magnus_color for clarity.

cg = cg.drop(['site', 'ply_count', 'round','game_order','event','date_played','white_rating_diff','white_title',
    'black_rating_diff','black_title', 'file_name', 'date_created','winner','winner_elo','loser','loser_elo',
    'utc_date','utc_time','winner_loser_elo_diff','eco','termination','variant'],axis=1)
cg = cg.rename(columns={'group_name_result':'magnus_result','group_name_color':'magnus_color'})
cg.head()

,game_id,white,black,result,white_elo,black_elo,time_control,magnus_color,magnus_result,datetime
0,b61b981c-e005-4064-8cc8-6feae75530b3,BabaRamdev,damnsaltythatsport,1-0,2739,2873,60+0,black,lost,2018.08.04 20:58:08
1,913cad09-25de-4c53-8937-17772cd05544,damnsaltythatsport,penguingim1,1-0,2863,2836,60+0,white,won,2018.08.04 20:56:59
2,62fcaebd-c829-4357-b9eb-1c1c373d6685,Hawk20,damnsaltythatsport,0-1,2677,2857,60+0,black,won,2018.08.04 20:54:40
3,f075f85f-bafc-4d21-8841-cb3017fbd43b,damnsaltythatsport,opperwezen,1-0,2848,2774,60+0,white,won,2018.08.04 20:52:19
4,bb6c0012-3863-4669-9c4e-9d230b720b04,caching,damnsaltythatsport,1-0,2770,2862,60+0,black,lost,2018.08.04 20:50:20


In [395]:
list(cg.columns.values)

['game_id',
 'white',
 'black',
 'result',
 'white_elo',
 'black_elo',
 'time_control',
 'magnus_color',
 'magnus_result',
 'datetime']

In [396]:
# Rearranging columns so white info is together and black info is together
cg = cg[['game_id','white_elo','white','result','black','black_elo','time_control','magnus_color','magnus_result','datetime']]
cg.head()

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime
0,b61b981c-e005-4064-8cc8-6feae75530b3,2739,BabaRamdev,1-0,damnsaltythatsport,2873,60+0,black,lost,2018.08.04 20:58:08
1,913cad09-25de-4c53-8937-17772cd05544,2863,damnsaltythatsport,1-0,penguingim1,2836,60+0,white,won,2018.08.04 20:56:59
2,62fcaebd-c829-4357-b9eb-1c1c373d6685,2677,Hawk20,0-1,damnsaltythatsport,2857,60+0,black,won,2018.08.04 20:54:40
3,f075f85f-bafc-4d21-8841-cb3017fbd43b,2848,damnsaltythatsport,1-0,opperwezen,2774,60+0,white,won,2018.08.04 20:52:19
4,bb6c0012-3863-4669-9c4e-9d230b720b04,2770,caching,1-0,damnsaltythatsport,2862,60+0,black,lost,2018.08.04 20:50:20


In [397]:
# Changing Lichess usernames to real names
# The encoding deals with foreign characters 
usernames = pd.read_csv('csv/user_names.csv',encoding="ISO-8859-1")
usernamedict = dict(zip(usernames.user_name,usernames.real_name))
usernamedict['DannyTheDonkey'] = 'Magnus Carlsen' # csv has 'DannytheDonkey' instead of 'DannyTheDonkey' so I'm fixing it using this
cg = cg.replace(usernamedict)
cg.head()

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime
0,b61b981c-e005-4064-8cc8-6feae75530b3,2739,Akshat Chandra,1-0,Magnus Carlsen,2873,60+0,black,lost,2018.08.04 20:58:08
1,913cad09-25de-4c53-8937-17772cd05544,2863,Magnus Carlsen,1-0,Andrew Tang,2836,60+0,white,won,2018.08.04 20:56:59
2,62fcaebd-c829-4357-b9eb-1c1c373d6685,2677,Hawk20,0-1,Magnus Carlsen,2857,60+0,black,won,2018.08.04 20:54:40
3,f075f85f-bafc-4d21-8841-cb3017fbd43b,2848,Magnus Carlsen,1-0,Vincent Rothuis,2774,60+0,white,won,2018.08.04 20:52:19
4,bb6c0012-3863-4669-9c4e-9d230b720b04,2770,caching,1-0,Magnus Carlsen,2862,60+0,black,lost,2018.08.04 20:50:20


In [398]:
# Using game_id to find Magnus's first moves from carlsen_games_moves.csv.
# It's a 1133609 * 64 spreadsheet so the next 3 code blocks takes a while. (30 seconds, 40 seconds, 90 seconds)
moves = pd.read_csv('csv/carlsen_games_moves.csv')

In [399]:
moves = pd.DataFrame(data=[moves.game_id,moves.move_no_pair,moves.player,moves.notation]).T # only taking the columns i need from the csv


In [400]:
moves = moves.replace(usernamedict) # usernames to real names
moves

,game_id,move_no_pair,player,notation
0,b61b981c-e005-4064-8cc8-6feae75530b3,1,Akshat Chandra,d4
1,b61b981c-e005-4064-8cc8-6feae75530b3,1,Magnus Carlsen,Nf6
2,b61b981c-e005-4064-8cc8-6feae75530b3,2,Akshat Chandra,c4
3,b61b981c-e005-4064-8cc8-6feae75530b3,2,Magnus Carlsen,e6
4,b61b981c-e005-4064-8cc8-6feae75530b3,3,Akshat Chandra,Nc3
...,...,...,...,...
1133604,26b967ce-c77f-4400-95d1-20ccc8468ded,57,Magnus Carlsen,Rg8
1133605,26b967ce-c77f-4400-95d1-20ccc8468ded,57,Jeffery Xiong,b2
1133606,26b967ce-c77f-4400-95d1-20ccc8468ded,58,Magnus Carlsen,Rxg6
1133607,26b967ce-c77f-4400-95d1-20ccc8468ded,58,Jeffery Xiong,Rh1


In [401]:
# first moves made by magnus
fmoves = moves.loc[(moves.move_no_pair==1) & (moves.player=='Magnus Carlsen')]
fmoves

,game_id,move_no_pair,player,notation
1,b61b981c-e005-4064-8cc8-6feae75530b3,1,Magnus Carlsen,Nf6
123,913cad09-25de-4c53-8937-17772cd05544,1,Magnus Carlsen,c3
180,62fcaebd-c829-4357-b9eb-1c1c373d6685,1,Magnus Carlsen,e6
323,f075f85f-bafc-4d21-8841-cb3017fbd43b,1,Magnus Carlsen,c4
437,bb6c0012-3863-4669-9c4e-9d230b720b04,1,Magnus Carlsen,e6
...,...,...,...,...
1133064,f976c4fb-5e47-4e30-9873-ad7329f23eec,1,Magnus Carlsen,Nf3
1133165,4b39774a-70aa-4079-b6c9-225a377e0635,1,Magnus Carlsen,c5
1133269,d1c00844-fdb6-4651-8a8d-d2256ef4f202,1,Magnus Carlsen,Nf3
1133357,47bdf460-ae8c-4d60-a996-8cf93c43203e,1,Magnus Carlsen,d5


In [402]:
# Merging on game_id to get the first moves
cg = cg.merge(fmoves,on='game_id')
cg

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime,move_no_pair,player,notation
0,b61b981c-e005-4064-8cc8-6feae75530b3,2739,Akshat Chandra,1-0,Magnus Carlsen,2873,60+0,black,lost,2018.08.04 20:58:08,1,Magnus Carlsen,Nf6
1,913cad09-25de-4c53-8937-17772cd05544,2863,Magnus Carlsen,1-0,Andrew Tang,2836,60+0,white,won,2018.08.04 20:56:59,1,Magnus Carlsen,c3
2,62fcaebd-c829-4357-b9eb-1c1c373d6685,2677,Hawk20,0-1,Magnus Carlsen,2857,60+0,black,won,2018.08.04 20:54:40,1,Magnus Carlsen,e6
3,f075f85f-bafc-4d21-8841-cb3017fbd43b,2848,Magnus Carlsen,1-0,Vincent Rothuis,2774,60+0,white,won,2018.08.04 20:52:19,1,Magnus Carlsen,c4
4,bb6c0012-3863-4669-9c4e-9d230b720b04,2770,caching,1-0,Magnus Carlsen,2862,60+0,black,lost,2018.08.04 20:50:20,1,Magnus Carlsen,e6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,f976c4fb-5e47-4e30-9873-ad7329f23eec,2881,Magnus Carlsen,0-1,Jeffery Xiong,2730,600+5,white,lost,2020.06.06 20:11:23,1,Magnus Carlsen,Nf3
13422,4b39774a-70aa-4079-b6c9-225a377e0635,2730,Jeffery Xiong,1/2-1/2,Magnus Carlsen,2881,600+5,black,draw,2020.06.06 19:39:07,1,Magnus Carlsen,c5
13423,d1c00844-fdb6-4651-8a8d-d2256ef4f202,2881,Magnus Carlsen,1-0,Jeffery Xiong,2730,600+5,white,won,2020.06.06 19:14:18,1,Magnus Carlsen,Nf3
13424,47bdf460-ae8c-4d60-a996-8cf93c43203e,2730,Jeffery Xiong,0-1,Magnus Carlsen,2881,600+5,black,won,2020.06.06 18:30:37,1,Magnus Carlsen,d5


In [403]:
# Sorting games by datetime
cg = cg.sort_values(by=['datetime'],ignore_index=True)
cg.head()

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime,move_no_pair,player,notation
0,298f0889-0249-4db0-ba19-f6483ec1e6de,2500,Magnus Carlsen,1-0,jsalomon,2532,60+0,black,lost,2017.12.14 21:00:02,1,Magnus Carlsen,d4
1,52d5e7da-22b9-422f-b695-fec4be9bedb1,2420,Yoseph Taher,1-0,Magnus Carlsen,2558,60+0,black,lost,2017.12.14 21:01:10,1,Magnus Carlsen,d5
2,30157fb0-b75e-437c-a426-15de66858bae,2754,gnejs,0-1,Magnus Carlsen,2493,60+0,black,lost,2017.12.14 21:04:13,1,Magnus Carlsen,d5
3,93fa1904-8b3f-47cb-a3b3-effd45ef270a,2566,Magnus Carlsen,1-0,Toivo Keinänen,2755,60+0,black,lost,2017.12.14 21:06:30,1,Magnus Carlsen,b3
4,5677b65e-8fa4-44a1-a64b-35067160a80c,2627,Magnus Carlsen,1-0,AusCub,2275,60+0,black,lost,2017.12.14 21:07:57,1,Magnus Carlsen,b3


In [404]:
# The first row shows that the magnus_result and magnus_color columns are incorrect so I'll fix them below.
# Fixing magnus_color: magwhite and magblack is full of "Magnus Carlsen" and NaN values. I merge those 2 dfs to make magcolor

magwhite = cg.loc[(cg.white=='Magnus Carlsen'),['white']]
magwhite.reset_index(inplace=True)
magblack = cg.loc[(cg.black=='Magnus Carlsen'),['black']]
magblack.reset_index(inplace=True)
magcolor = magwhite.merge(magblack,how='outer',on='index',sort=True)

magcolor['magnus_color'] = (magcolor.white=='Magnus Carlsen') # full of booleans where True means Magnus was white
magcolor = magcolor.replace({True:'white',False:'black'})
cg.magnus_color = magcolor.magnus_color
cg

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime,move_no_pair,player,notation
0,298f0889-0249-4db0-ba19-f6483ec1e6de,2500,Magnus Carlsen,1-0,jsalomon,2532,60+0,white,lost,2017.12.14 21:00:02,1,Magnus Carlsen,d4
1,52d5e7da-22b9-422f-b695-fec4be9bedb1,2420,Yoseph Taher,1-0,Magnus Carlsen,2558,60+0,black,lost,2017.12.14 21:01:10,1,Magnus Carlsen,d5
2,30157fb0-b75e-437c-a426-15de66858bae,2754,gnejs,0-1,Magnus Carlsen,2493,60+0,black,lost,2017.12.14 21:04:13,1,Magnus Carlsen,d5
3,93fa1904-8b3f-47cb-a3b3-effd45ef270a,2566,Magnus Carlsen,1-0,Toivo Keinänen,2755,60+0,white,lost,2017.12.14 21:06:30,1,Magnus Carlsen,b3
4,5677b65e-8fa4-44a1-a64b-35067160a80c,2627,Magnus Carlsen,1-0,AusCub,2275,60+0,white,lost,2017.12.14 21:07:57,1,Magnus Carlsen,b3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13421,43c6f315-c304-406b-b30b-92a803a77d92,3202,Magnus Carlsen,1-0,Aleksei Sarana,2984,180+0,white,won,2021.12.23 23:07:55,1,Magnus Carlsen,e4
13422,df129181-068c-4a2d-ba53-e98b679e0caa,2982,Aleksei Sarana,0-1,Magnus Carlsen,3204,180+0,black,won,2021.12.23 23:10:43,1,Magnus Carlsen,Nf6
13423,6e6c9a7f-69e5-42ac-b4ab-b5671af9828f,3207,Magnus Carlsen,1-0,Aleksei Sarana,2979,180+0,white,won,2021.12.23 23:15:48,1,Magnus Carlsen,e4
13424,7a790feb-3aa2-43da-88f8-d14f6cd82867,2977,Aleksei Sarana,0-1,Magnus Carlsen,3209,180+0,black,won,2021.12.23 23:21:54,1,Magnus Carlsen,Nf6


In [405]:
# fixing magnus_result
# 1-0 means white wins, 0-1 means black wins and 1/2-1/2 means a draw.
def result(df):
    if (df.white=='Magnus Carlsen') & (df.result[:2]=='1-') or (df.black=='Magnus Carlsen') & (df.result[:2]=='0-'):
        return 'won'
    if df.result=='1/2-1/2':
        return 'draw'
    else:
        return 'lost'

cg.magnus_result = cg.apply(result,axis=1)
cg.head()

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime,move_no_pair,player,notation
0,298f0889-0249-4db0-ba19-f6483ec1e6de,2500,Magnus Carlsen,1-0,jsalomon,2532,60+0,white,won,2017.12.14 21:00:02,1,Magnus Carlsen,d4
1,52d5e7da-22b9-422f-b695-fec4be9bedb1,2420,Yoseph Taher,1-0,Magnus Carlsen,2558,60+0,black,lost,2017.12.14 21:01:10,1,Magnus Carlsen,d5
2,30157fb0-b75e-437c-a426-15de66858bae,2754,gnejs,0-1,Magnus Carlsen,2493,60+0,black,won,2017.12.14 21:04:13,1,Magnus Carlsen,d5
3,93fa1904-8b3f-47cb-a3b3-effd45ef270a,2566,Magnus Carlsen,1-0,Toivo Keinänen,2755,60+0,white,won,2017.12.14 21:06:30,1,Magnus Carlsen,b3
4,5677b65e-8fa4-44a1-a64b-35067160a80c,2627,Magnus Carlsen,1-0,AusCub,2275,60+0,white,won,2017.12.14 21:07:57,1,Magnus Carlsen,b3


In [406]:
# Changing time_control from seconds+seconds to minutes+seconds because minutes+seconds is convention

def timeconvert(df): # games under one minute have a decimal so I differentiate to avoid unnecessary decimals.
    minutes = int(df.time_control[:df.time_control.index('+')])/60
    if minutes==int(minutes): 
        return str(int(minutes)) + df.time_control[df.time_control.index('+'):]
    return str(minutes) + df.time_control[df.time_control.index('+'):]

cg.time_control = cg.apply(timeconvert,axis=1)

In [407]:
# Removing the move_no_pair and player column because it's not useful anymore. Changing 'notation' to first_move for clarity.
# Final version shown below.
cg = cg.drop(['move_no_pair','player'],axis=1)
cg = cg.rename(columns={'notation':'first_move'})
cg.head()

,game_id,white_elo,white,result,black,black_elo,time_control,magnus_color,magnus_result,datetime,first_move
0,298f0889-0249-4db0-ba19-f6483ec1e6de,2500,Magnus Carlsen,1-0,jsalomon,2532,1+0,white,won,2017.12.14 21:00:02,d4
1,52d5e7da-22b9-422f-b695-fec4be9bedb1,2420,Yoseph Taher,1-0,Magnus Carlsen,2558,1+0,black,lost,2017.12.14 21:01:10,d5
2,30157fb0-b75e-437c-a426-15de66858bae,2754,gnejs,0-1,Magnus Carlsen,2493,1+0,black,won,2017.12.14 21:04:13,d5
3,93fa1904-8b3f-47cb-a3b3-effd45ef270a,2566,Magnus Carlsen,1-0,Toivo Keinänen,2755,1+0,white,won,2017.12.14 21:06:30,b3
4,5677b65e-8fa4-44a1-a64b-35067160a80c,2627,Magnus Carlsen,1-0,AusCub,2275,1+0,white,won,2017.12.14 21:07:57,b3


In [408]:
# Used to access cg in another notebook 
%store cg

Stored 'cg' (DataFrame)


In [409]:
# In the few code blocks, I'm creating a dataframe where Magnus has his own column and the opponent has their own column, instead of black and white
def mergeelo(row):
    return row.white_elo + row.black_elo

In [410]:
# Similar to the creation of the magnus_color column from before except I change the NaNs to 0 and
# the elo column is the sum of white_elo and black_elo in which one of them is 0 and the other is Magnus's elo.
magnuswhiteelo = cg.loc[(cg.white=='Magnus Carlsen'),['white_elo']]
magnuswhiteelo.reset_index(inplace=True)
magnusblackelo = cg.loc[(cg.black=='Magnus Carlsen'),['black_elo']]
magnusblackelo.reset_index(inplace=True)
magnuselo = magnuswhiteelo.merge(magnusblackelo,how='outer',on='index',sort=True)
magnuselo = magnuselo.fillna(0)
magnuselo['elo']= magnuselo.apply(lambda row: mergeelo(row),axis=1)
magnuselo

,index,white_elo,black_elo,elo
0,0,2500.0,0.0,2500.0
1,1,0.0,2558.0,2558.0
2,2,0.0,2493.0,2493.0
3,3,2566.0,0.0,2566.0
4,4,2627.0,0.0,2627.0
...,...,...,...,...
13421,13421,3202.0,0.0,3202.0
13422,13422,0.0,3204.0,3204.0
13423,13423,3207.0,0.0,3207.0
13424,13424,0.0,3209.0,3209.0


In [411]:
# Doing the same thing to opponent elo now
opponentwhiteelo = cg.loc[(cg.white!='Magnus Carlsen'),['white_elo']]
opponentwhiteelo.reset_index(inplace=True)
opponentblackelo = cg.loc[(cg.black!='Magnus Carlsen'),['black_elo']]
opponentblackelo.reset_index(inplace=True)
opponentelo = opponentwhiteelo.merge(opponentblackelo,how='outer',on='index',sort=True)
opponentelo = opponentelo.fillna(0)
opponentelo['elo']= opponentelo.apply(lambda row: mergeelo(row),axis=1)
opponentelo

,index,white_elo,black_elo,elo
0,0,0.0,2532.0,2532.0
1,1,2420.0,0.0,2420.0
2,2,2754.0,0.0,2754.0
3,3,0.0,2755.0,2755.0
4,4,0.0,2275.0,2275.0
...,...,...,...,...
13421,13421,0.0,2984.0,2984.0
13422,13422,2982.0,0.0,2982.0
13423,13423,0.0,2979.0,2979.0
13424,13424,2977.0,0.0,2977.0


In [412]:
# combining the information from the two dataframes above
magoppres = pd.DataFrame(data=(magnuselo.elo,opponentelo.elo,cg.magnus_result)).T
magoppres.columns = ['mag_elo','opp_elo','magnus_result']
magoppres

,mag_elo,opp_elo,magnus_result
0,2500.0,2532.0,won
1,2558.0,2420.0,lost
2,2493.0,2754.0,won
3,2566.0,2755.0,won
4,2627.0,2275.0,won
...,...,...,...
13421,3202.0,2984.0,won
13422,3204.0,2982.0,won
13423,3207.0,2979.0,won
13424,3209.0,2977.0,won


In [413]:
# mg short for magnus games but mainly means that the dataframe has a magnus column and a opponent column rather than black/white
mg = pd.DataFrame(data=(cg.magnus_color,magoppres.mag_elo,magoppres.opp_elo, magoppres.magnus_result,cg.time_control,cg.datetime,cg.first_move)).T
%store mg

Stored 'mg' (DataFrame)
